This notebook shows the basic steps that are involved in movement detection. Much of what's in here can be encaspulated by higher-order functions or classes, but if you want to have full control over things, have a look at the following.

In [2]:
import birdwatcher as bw

## Create a background subtractor object with suitable parameters

Create a background subtractor object with default parameters. This object basically encapsulates the algorithm that determines the background from a history of images, and uses that to find what is not background in the current image. See opencv page for more info on the algorithm: https://docs.opencv.org/3.4/d7/d7b/classcv_1_1BackgroundSubtractorMOG2.html

In [2]:
bgs = bw.BackgroundSubtractorMOG2()

What are the parameters?

In [3]:
bgs.get_params()

{'History': 5,
 'ComplexityReductionThreshold': 0.05,
 'BackgroundRatio': 0.1,
 'NMixtures': 7,
 'VarInit': 15,
 'VarMin': 4,
 'VarMax': 75,
 'VarThreshold': 10,
 'VarThresholdGen': 9,
 'DetectShadows': False,
 'ShadowThreshold': 0.5,
 'ShadowValue': 127}

You can use non-default paramaters by specifying them at intstantiation.

In [4]:
bgs = bw.BackgroundSubtractorMOG2(VarThreshold=70, NMixtures=8, History=3)

In [5]:
bgs.get_params()

{'History': 3,
 'ComplexityReductionThreshold': 0.05,
 'BackgroundRatio': 0.1,
 'NMixtures': 8,
 'VarInit': 15,
 'VarMin': 4,
 'VarMax': 75,
 'VarThreshold': 70,
 'VarThresholdGen': 9,
 'DetectShadows': False,
 'ShadowThreshold': 0.5,
 'ShadowValue': 127}

## Create a video object (see separate notebook on this)

In [6]:
vf = bw.testvideosmall()

## Do movement detection by hand based on this background subtractor

This can be done by setting up a pipe line that generates and processes image sequences

First set up a frame generator that produces gray frames from the color video file object:

In [7]:
frames = vf.iter_frames(color=False)

Now feed that to the background subtractor that we created above. It returns another frame generator.

In [8]:
frames = bgs.iter_apply(frames, learningRate=-1)

We now have a frame generator that produces foreground mask frames. Let's get rid of some noise. (look at MorphologyEx page of opencv for what this does: https://docs.opencv.org/3.4/d9/d61/tutorial_py_morphological_ops.html)

In [9]:
frames = frames.morphologyex(morphtype='open', kernelsize=2)

We are ready to start running the whole frame pipe line and save results as a video for inspection.

In [10]:
frames = frames.tovideo('test_MOG2.mp4', framerate=vf.avgframerate)

This whole pipeline can also be shortened like so:

In [11]:
frames = (bgs.iter_apply(vf.iter_frames(color=False), learningRate=-1)
                        .morphologyex(morphtype='open', kernelsize=2)
                        .tovideo('test_MOG2.mp4', framerate=vf.avgframerate))

## Create a coordinate array for storage of the results

We wrote the movement detection results (suprathreshold pixels) above to a video so that we could view the results. However, if you want to save the results for further analyses it is much better to save tham as a *coordinate array*.

A coordinate array really is just a Darr ragged array (see separate library). This makes it easy to read the data in other environments, e.g. R. We save some metadata so that we later know what we did. Instead of saving the frames to video, we now detect non-zero pixel (i.e. foreground) coordinates, and save that to the coordinate array.

First set up analysis pipe line:

In [12]:
coordinates = (bgs.iter_apply(vf.iter_frames(color=False), learningRate=-1)
                             .morphologyex(morphtype='open', kernelsize=2)
                             .find_nonzero())

Create an empty CoordinateArray object to save to:

In [13]:
coordsarray = bw.create_coordarray('testcoords.darr', 
                                   framewidth=vf.framewidth,
                                   frameheight=vf.frameheight,
                                   metadata={'bgsparams': bgs.get_params(),
                                             'morphologyex': ('open', 2),
                                             'learningrate': -1}, 
                                   overwrite=True)

In [14]:
coordsarray.metadata

{'bgsparams': {'BackgroundRatio': 0.1, 'ComplexityReductionThreshold': 0.05, 'DetectShadows': False, 'History': 3, 'NMixtures': 8, 'ShadowThreshold': 0.5, 'ShadowValue': 127, 'VarInit': 15, 'VarMax': 75, 'VarMin': 4, 'VarThreshold': 70, 'VarThresholdGen': 9}, 'birdwatcher_version': '0.1.0+236.g3811bfc.dirty', 'frameheight': 720, 'framewidth': 1280, 'learningrate': -1, 'morphologyex': ['open', 2]}

Now save pipeline to array:

In [15]:
%%time
for c in coordinates:
    coordsarray.append(c)

Wall time: 16.9 s


## Accessing coordinate arrays

The coordinate array can be accessed in other python sessions like so:

In [16]:
coordsarray = bw.CoordinateArrays('testcoords.darr')

In [17]:
coordsarray[100] # coordinates of the 101th frame 

array([[494, 390],
       [495, 390],
       [491, 391],
       [492, 391],
       [494, 391],
       [495, 391],
       [491, 392],
       [492, 392],
       [493, 392],
       [494, 392],
       [495, 392],
       [496, 392],
       [493, 393],
       [494, 393],
       [495, 393],
       [496, 393],
       [495, 394],
       [496, 394],
       [495, 395],
       [496, 395],
       [491, 397],
       [492, 397],
       [494, 397],
       [495, 397],
       [491, 398],
       [492, 398],
       [493, 398],
       [494, 398],
       [495, 398],
       [492, 399],
       [493, 399],
       [490, 400],
       [491, 400],
       [492, 400],
       [493, 400],
       [487, 401],
       [488, 401],
       [490, 401],
       [491, 401],
       [492, 401],
       [493, 401],
       [482, 402],
       [483, 402],
       [484, 402],
       [487, 402],
       [488, 402],
       [482, 403],
       [483, 403],
       [484, 403],
       [481, 404],
       [482, 404],
       [483, 404],
       [484,

## Take it together and look at a range of parameters

As an example we'll look at a range of history settings. Note that we do not have to run the analysis pipe line twice in order to get both coordinate results and a video. We just create a coordinate array first. This can the be saved as a video for inspection.

In [3]:
%%time
vf = bw.testvideosmall()
for history in (2,3,4):
    bgs = bw.BackgroundSubtractorMOG2(History=history, VarThreshold=50)
    coordinates = (bgs.iter_apply(vf.iter_frames(color=False), learningRate=-1)
                         .morphologyex(morphtype='open', kernelsize=2)
                         .find_nonzero())
    basefilename = f'testcoords_hist{history}'
    coordsarray = bw.create_coordarray(f'{basefilename}.darr', 
                                       framewidth=vf.framewidth,
                                       frameheight=vf.frameheight, 
                                       metadata={'bgsparams': bgs.get_params(),
                                                 'morphologyex': ('open', 2),
                                                 'learningrate': -1}, 
                                       overwrite=True)
    for c in coordinates:
        coordsarray.append(c)
    coordsarray.tovideo(f'{basefilename}.mp4', framerate=vf.avgframerate)

CPU times: user 1min 19s, sys: 12.7 s, total: 1min 32s
Wall time: 29.5 s
